In [1]:
# Import required libraries  
import os  
from openai import OpenAI
from dotenv import load_dotenv  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient 
from azure.search.documents.models import (
    VectorizedQuery,
    VectorFilterMode,    
)
from backend.tools.searchtool import Search
  
# Configure environment variables  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
credential = AzureKeyCredential(key)
openai_client = OpenAI()
openai_model = os.getenv("OPENAI_MODEL")

search_client: Search = Search('sales_vector_index')  # get instance of search to query corpus using the name of the index

creating search client with - brandspeak-index


In [2]:
def get_embedding(text, model='text-embedding-ada-002'):
   text = text.replace("\n", " ")
   return openai_client.embeddings.create(input = [text], model=model).data[0].embedding

In [6]:
# Pure Vector Search
query = "satya keynote nvidia"  
  
results = search_client.search_hybrid(query)  
  

Starting Search - satya keynote nvidia
Got embedding
Got result
[SOURCEFILE:  Full Keynote Satya Nadella at Microsoft Ignite 2023.txt]One year ago, at the dawn of a new age, we took the first bold step into a world of unlimited possibilities. We're going to get to think about how we use our imagination to solve some of the biggest problems in the world. It allows for the patient and doctor to be a patient or a doctor like it used to be. If AI as your co-pilot, you are supercharging innovation. It's like more than my mind. It's magic. You built new solutions and inspired us to imagine more. In our journey of Generative AI, we have over 700 use cases across all divisions and functions, so we really see a big impact. Co-pilot makes day-to-day life easier so you can spend your time thinking how I want to improve the experience for people learning a language. This technology has the potential to completely reimagine the way every single student learns in the world. It enables all people to 

In [ ]:
# Hybrid
query = "bedrock"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
  
vector_query = VectorizedQuery(vector=get_embedding(query), k_nearest_neighbors=3, fields="contentVector")
  
category = "AWS"

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter=f"category eq '{category}'",
    #filter="category eq 'Microsoft'",
    select=["title", "content", "category", "spr"],
)
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  
    print(f"SPR: {result['spr']}\n")  